# Importamos librerias requeridas

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
def norm_list(lista: list):
    lista = str(lista)
    lista = lista.replace('[', '')
    lista = lista.replace("'", '')
    lista = lista.replace(']', '')
    lista = lista.replace(',', '')
    return lista

# Creamos un dataframe con lo requerido para el sistema de recomendacion


In [3]:
file_csv = 'D:/DataScience/PI/backup/data_definitiva/data_movies_full.csv'
csv = pd.read_csv(file_csv, index_col=0)

# Cargamos la base de datos limpia para extraer la informacion requerida

In [4]:
data = csv
data.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,id_coleccion,nombre_coleccion,id_companie,companie,iso_3166_1,country,iso_639_1,languaje,name_actor,name_director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,3.0,Pixar Animation Studios,US,United States of America,en,English,Tom Hanks,John Lasseter
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,3.0,Pixar Animation Studios,US,United States of America,en,English,Tim Allen,John Lasseter
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,3.0,Pixar Animation Studios,US,United States of America,en,English,Don Rickles,John Lasseter
3,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,3.0,Pixar Animation Studios,US,United States of America,en,English,Jim Varney,John Lasseter
4,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,3.0,Pixar Animation Studios,US,United States of America,en,English,Wallace Shawn,John Lasseter


Tomamos los campos con que trabajaremos para el algoritmo

In [5]:
data= data[['id','overview', 'title', 'genero', 'name_actor']]
data.drop_duplicates(inplace=True)
data.head()

,id,overview,title,genero,name_actor
0,862,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation,Tom Hanks
1,862,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation,Tim Allen
2,862,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation,Don Rickles
3,862,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation,Jim Varney
4,862,"Led by Woody, Andy's toys live happily in his ...",Toy Story,Animation,Wallace Shawn


# Creamos nuevos dataframes solo con el id, nombre de actos y los generos de las peliculas

Tomamos todos los generos y actores correspondientes a cada pelicula y los agregramos a un diccionario con el respectivo id de la pelicula

- Generos

In [6]:
aux_gen = data[['id', 'genero']]
aux_gen.drop_duplicates(inplace=True)
dict_list_genres = aux_gen.to_dict('records')
dict_list_genres[:5]

[{'id': 862, 'genero': 'Animation'},
 {'id': 862, 'genero': 'Comedy'},
 {'id': 862, 'genero': 'Family'},
 {'id': 8844, 'genero': 'Adventure'},
 {'id': 8844, 'genero': 'Fantasy'}]

- Nombre de actores


In [7]:
aux_act = data[['id', 'name_actor']]
aux_act.drop_duplicates(inplace=True)
dict_list_actors = aux_act.to_dict('records')
dict_list_actors[:5]

[{'id': 862, 'name_actor': 'Tom Hanks'},
 {'id': 862, 'name_actor': 'Tim Allen'},
 {'id': 862, 'name_actor': 'Don Rickles'},
 {'id': 862, 'name_actor': 'Jim Varney'},
 {'id': 862, 'name_actor': 'Wallace Shawn'}]

Unificamos los diccionarios con solo una clave y el valor son todos los generos o actores correspondientes a la pelicula

- Generos

In [ ]:
nuevo_dicc_genres = {}
for diccionario in dict_list_genres:
    clave = diccionario['id']
    valor = diccionario['genero']
    
    if clave in nuevo_dicc_genres:
        nuevo_dicc_genres[clave].append(valor)
    else:
        nuevo_dicc_genres[clave] = [valor]

nuevo_dicc_genres

- Nombre actores

In [ ]:
nuevo_dicc_act = {}
for diccionario in dict_list_actors:
    clave = diccionario['id']
    valor = diccionario['name_actor']
    
    if clave in nuevo_dicc_act:
        nuevo_dicc_act[clave].append(valor)
    else:
        nuevo_dicc_act[clave] = [valor]

nuevo_dicc_act

# Tratamiento de las lista que contienen los generos de las peliculas y los actores

Creamos dataframes con la columna de actores y generos

- Generos

In [10]:
for key in nuevo_dicc_genres:
    nuevo_dicc_genres[key] = str(nuevo_dicc_genres[key])

df_genres = pd.DataFrame(nuevo_dicc_genres.items(), columns=['id', 'generos'])
df_genres.head()

,id,generos
0,862,"['Animation', 'Comedy', 'Family']"
1,8844,"['Adventure', 'Fantasy', 'Family']"
2,15602,"['Romance', 'Comedy']"
3,31357,"['Comedy', 'Drama', 'Romance']"
4,11862,['Comedy']


- Actores

In [11]:
for key in nuevo_dicc_act:
    nuevo_dicc_act[key] = str(nuevo_dicc_act[key])

df_act = pd.DataFrame(nuevo_dicc_act.items(), columns=['id', 'actors'])
df_act.head()

,id,actors
0,862,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim..."
1,8844,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."
2,15602,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret..."
3,31357,"['Whitney Houston', 'Angela Bassett', 'Loretta..."
4,11862,"['Steve Martin', 'Diane Keaton', 'Martin Short..."


Convertimos las lista en str para eliminar los corchetes, comas y comillas y poder hacer el tratamiento de este como un string completo 

- Genero

In [12]:
df_genres['generos'] =  df_genres.generos.apply(norm_list)
df_genres.head()

,id,generos
0,862,Animation Comedy Family
1,8844,Adventure Fantasy Family
2,15602,Romance Comedy
3,31357,Comedy Drama Romance
4,11862,Comedy


- Actores

In [13]:
df_act['actors'] =  df_act.actors.apply(norm_list)
df_act.head()

,id,actors
0,862,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...
1,8844,Robin Williams Jonathan Hyde Kirsten Dunst Bra...
2,15602,Walter Matthau Jack Lemmon Ann-Margret Sophia ...
3,31357,Whitney Houston Angela Bassett Loretta Devine ...
4,11862,Steve Martin Diane Keaton Martin Short Kimberl...


# Creamos un nuevo dataframe con el id, titulo, lista de actores y lista de generos

Creamos un nuevo dataframe con los titulos de las peliculas

In [14]:
df_title = data[['id', 'title']]
df_title.drop_duplicates(inplace=True)
df_title.head()

,id,title
0,862,Toy Story
39,8844,Jumanji
507,15602,Grumpier Old Men
535,31357,Waiting to Exhale
565,11862,Father of the Bride Part II


Creamos un nuevo dataframe con los overviews de las peliculas

In [15]:
df_oview = data[['id', 'overview']]
df_oview.drop_duplicates(inplace=True)
df_oview.head()

,id,overview
0,862,"Led by Woody, Andy's toys live happily in his ..."
39,8844,When siblings Judy and Peter discover an encha...
507,15602,A family wedding reignites the ancient feud be...
535,31357,"Cheated on, mistreated and stepped on, the wom..."
565,11862,Just when George Banks has recovered from his ...


Hacemos un merge entre los dataframe de titulos, lista de actores y generos

In [16]:
dataset = pd.merge(df_title,df_genres, how='left', on='id')
dataset = pd.merge(dataset, df_act, how='left', on='id')
dataset

,id,title,generos,actors
0,862,Toy Story,Animation Comedy Family,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...
1,8844,Jumanji,Adventure Fantasy Family,Robin Williams Jonathan Hyde Kirsten Dunst Bra...
2,15602,Grumpier Old Men,Romance Comedy,Walter Matthau Jack Lemmon Ann-Margret Sophia ...
3,31357,Waiting to Exhale,Comedy Drama Romance,Whitney Houston Angela Bassett Loretta Devine ...
4,11862,Father of the Bride Part II,Comedy,Steve Martin Diane Keaton Martin Short Kimberl...
...,...,...,...,...
45341,30840,Robin Hood,Drama Action Romance,Patrick Bergin Uma Thurman David Morrissey Jür...
45342,111109,Century of Birthing,Drama,Angel Aquino Perry Dizon Hazel Orencio Joel To...
45343,67758,Betrayal,Action Drama Thriller,Erika Eleniak Adam Baldwin Julie du Page James...
45344,227506,Satan Triumphant,nan,Iwan Mosschuchin Nathalie Lissenko Pavel Pavlo...


Creo un dataset con la informacion anterior y adicionalmente el overview de las peliculas

In [17]:
dataset_2 = pd.merge(dataset, df_oview, how='left', on='id')
dataset_2.head()

,id,title,generos,actors,overview
0,862,Toy Story,Animation Comedy Family,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,Adventure Fantasy Family,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,Romance Comedy,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,Comedy Drama Romance,Whitney Houston Angela Bassett Loretta Devine ...,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Comedy,Steve Martin Diane Keaton Martin Short Kimberl...,Just when George Banks has recovered from his ...


# finalmente creamos un campo con la union de los campos

- Actores y generos

In [18]:
dataset['tags'] = dataset.actors + ' ' + dataset.generos
dataset.drop(columns=['actors', 'generos'], inplace=True)
dataset

,id,title,tags
0,862,Toy Story,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...
1,8844,Jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Bra...
2,15602,Grumpier Old Men,Walter Matthau Jack Lemmon Ann-Margret Sophia ...
3,31357,Waiting to Exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,11862,Father of the Bride Part II,Steve Martin Diane Keaton Martin Short Kimberl...
...,...,...,...
45341,30840,Robin Hood,Patrick Bergin Uma Thurman David Morrissey Jür...
45342,111109,Century of Birthing,Angel Aquino Perry Dizon Hazel Orencio Joel To...
45343,67758,Betrayal,Erika Eleniak Adam Baldwin Julie du Page James...
45344,227506,Satan Triumphant,Iwan Mosschuchin Nathalie Lissenko Pavel Pavlo...


- Actores, generos y overviews

In [19]:
dataset_2['tags'] = dataset_2.actors + ' ' + dataset_2.generos + ' ' + dataset_2.overview
dataset_2.drop(columns=['actors', 'generos', 'overview'], inplace=True)
dataset_2

,id,title,tags
0,862,Toy Story,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...
1,8844,Jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Bra...
2,15602,Grumpier Old Men,Walter Matthau Jack Lemmon Ann-Margret Sophia ...
3,31357,Waiting to Exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,11862,Father of the Bride Part II,Steve Martin Diane Keaton Martin Short Kimberl...
...,...,...,...
45341,30840,Robin Hood,Patrick Bergin Uma Thurman David Morrissey Jür...
45342,111109,Century of Birthing,Angel Aquino Perry Dizon Hazel Orencio Joel To...
45343,67758,Betrayal,Erika Eleniak Adam Baldwin Julie du Page James...
45344,227506,Satan Triumphant,Iwan Mosschuchin Nathalie Lissenko Pavel Pavlo...


# Se ponen las columnas tags y title en minusculas


In [20]:
# dataset
dataset['tags'] = dataset['tags'].str.lower()
dataset['title'] = dataset['title'].str.lower()

# dataset_2
dataset_2['tags'] = dataset_2['tags'].str.lower()
dataset_2['title'] = dataset_2['title'].str.lower()

In [21]:
dataset.head()

,id,title,tags
0,862,toy story,tom hanks tim allen don rickles jim varney wal...
1,8844,jumanji,robin williams jonathan hyde kirsten dunst bra...
2,15602,grumpier old men,walter matthau jack lemmon ann-margret sophia ...
3,31357,waiting to exhale,whitney houston angela bassett loretta devine ...
4,11862,father of the bride part ii,steve martin diane keaton martin short kimberl...


In [22]:
dataset_2.head()

,id,title,tags
0,862,toy story,tom hanks tim allen don rickles jim varney wal...
1,8844,jumanji,robin williams jonathan hyde kirsten dunst bra...
2,15602,grumpier old men,walter matthau jack lemmon ann-margret sophia ...
3,31357,waiting to exhale,whitney houston angela bassett loretta devine ...
4,11862,father of the bride part ii,steve martin diane keaton martin short kimberl...


# Exportamos los dataset a csv

In [23]:
dataset.to_csv('dataset.csv')
dataset_2.to_csv('dataset_2.csv')

Creamos un dataset con la union de title y tags

In [7]:
data = pd.read_csv('dataset_2.csv', index_col=0)
data.head()

,id,title,tags
0,862,toy story,tom hanks tim allen don rickles jim varney wal...
1,8844,jumanji,robin williams jonathan hyde kirsten dunst bra...
2,15602,grumpier old men,walter matthau jack lemmon ann-margret sophia ...
3,31357,waiting to exhale,whitney houston angela bassett loretta devine ...
4,11862,father of the bride part ii,steve martin diane keaton martin short kimberl...


In [8]:
data.tags = data.title + ' ' + data.tags
data.head()

,id,title,tags
0,862,toy story,toy story tom hanks tim allen don rickles jim ...
1,8844,jumanji,jumanji robin williams jonathan hyde kirsten d...
2,15602,grumpier old men,grumpier old men walter matthau jack lemmon an...
3,31357,waiting to exhale,waiting to exhale whitney houston angela basse...
4,11862,father of the bride part ii,father of the bride part ii steve martin diane...


In [11]:
data.to_csv('dataset_3.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45346 entries, 0 to 45345
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45346 non-null  int64 
 1   title   45346 non-null  object
 2   tags    44405 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB
